# Spectral and total absorption rate (ExoPhot Working Group 4)

**1) Load libraries and functions to compute spectral absorption rate**

Load libraries and functions

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from os import walk
from pathlib import Path
import spectral_overlap_functions as sof # .py file with functions needed to quantify spectral overlap

ModuleNotFoundError: No module named 'spectral_overlap_functions'

Define functions that will be handy

In [ ]:
def plot_spectra(wl, sigma_int, spec_abs_rate_int, abs_rate, ax):
    """Helper function to plot absorption cross section
    and spectral absorption rate"""
    
    ax.plot(wl, sigma_int)
    ax1=ax.twinx()
    
    ax1.plot(wl, spec_abs_rate_int, c = 'tab:orange')
    ax.set_ylim(0,None)
    ax.autoscale(enable=True, axis='x', tight=True)
    ax1.set_ylim(0,None)
    ax1.autoscale(enable=True, axis='x', tight=True)
    ax.ticklabel_format(axis='y', style='sci', scilimits=(-1, 1), useOffset=False)
    ax1.ticklabel_format(axis='y', style='sci', scilimits=(-1, 1), useOffset=False)
    
    ax.set_title(pigm[:-4] + ' in \n' + atmos[:-4])
    ax.set_ylabel('Absorption \n Cross section (cm$^{-2}$)',color = 'tab:blue')
    ax1.set_ylabel('Spectral Absorption \n Rate ($\AA^{-1}$ s$^{-1}$)',color = 'tab:orange')
    ax.set_xlabel('Wavelength ($\AA$)')    
    
    abs_str = '{:.2f}'.format(abs_rate)
    ax.text(0.01, 0.85, 'Abs. Rate ($\gamma_t$):\n ' + abs_str + ' s$^{-1}$', horizontalalignment='left', 
         verticalalignment='center',color= 'gray', transform=ax.transAxes)


**2) Example of usage for a single combination of star, atmosphere and pigment**

In [ ]:
# Exoplanet system parameters (Sun/Earth pair)
Rs = 696340 # stellar radius in km
sma = 1.496e8 # Exoplanet orbital semi-major axis in km

# specify file names
atmos = 'atm_g2v_hzm_os1.txt'
star = 'SED_g2v.txt'
pigm = 'ChlA.txt'

# folders containing input files
folders = ['input/stellar/', 'input/atmosphere/', 'input/pigments/']

# read files to retrieve spectra
flux, trans, epsilon = sof.read_files(folders[0] + star, folders[1] + atmos, folders[2] + pigm)

# compute spectral and total absorption rates and ppfd
spectra, rates = sof.spectral_overlap(flux, trans, epsilon, Rs, sma)

# save results
sof.save_results(star, atmos, pigm, Rs, sma, S, spectra, rates)

# plot spectra
wl, F, T, sigma, rho_l = np.hsplit(spectra, spectra.shape[1])

fig, ax = plt.subplots(1,1,figsize=(5,3.5))
plot_spectra(wl, sigma, rho_l, rates[0], ax)

plt.tight_layout()
#plt.savefig('spectral_overlap_plot.jpg', dpi = 300)
plt.show()   


**3) Example of usage for several combinations of stars, atmospheres and pigments**

Retrieve names of files with star flux, atmosphere transmitance and pigment absorption spectra

In [ ]:
# --------------------------------------------------------------------
# Lists of atmospheres, pigments and stellar models files
# --------------------------------------------------------------------

# folders containing input files
folders = ['input/stellar/', 'input/atmosphere/', 'input/pigments/']

# Read and store the names of all files
_, _, names_star = next(walk(folders[0]))
_, _, names_atmos = next(walk(folders[1]))
_, _, names_pigm = next(walk(folders[2]))

# check file names
names_atmos, names_star, names_pigm

In [ ]:
# load auxiliary file containing solar radii (Rs) and habitable zone (hz) boundaries (inner, mid, outer) for different solar spectral types
df_exo = pd.read_csv('input/exosys_params.csv', comment = '#')
df_exo

In [ ]:
# iterate through atmosphere and pigment file names:
r1 = range(len(names_atmos))
r2 = range(len(names_pigm))

fig, ax = plt.subplots(len(names_atmos),len(names_pigm),figsize=(12,7))

for ii, jj in itertools.product(r1,r2): # this is a nested loop
    # select atmosphere and pigment file names
    atmos = names_atmos[ii]
    pigm =  names_pigm[jj]
    
    # the stellar model file and the exoplanet system parameters (Rs and sma) depend on the atmosphere file
    Stype, Rs, sma, S = sof.get_exo_params(atmos, df_exo)
    star = 'SED_' + Stype + '.txt' 
    
    # read files to retrieve spectra
    flux, trans, epsilon = sof.read_files(folders[0] + star, folders[1] + atmos, folders[2] + pigm)

    # compute spectral and total absorption rates and ppfd
    spectra, rates = sof.spectral_overlap(flux, trans, epsilon, Rs, sma)

    # save results
    sof.save_results(star, atmos, pigm, Rs, sma, S, spectra, rates)
        
    # plot spectra
    wl, F, T, sigma, rho_l = np.hsplit(spectra, spectra.shape[1])
    plot_spectra(wl, sigma, rho_l, rates[0], ax[ii,jj])

plt.tight_layout()
#plt.savefig('spectral_overlap_plot_multiple.jpg', dpi = 300)
plt.show()  
